In [45]:
# %% 
from qdrant_client import QdrantClient, models

QDRANT_URL = "http://localhost:6333"
COLLECTION_NAME = "cw04_hybrid_rerank"
DENSE_VECTOR_SIZE = 4096

client = QdrantClient(url=QDRANT_URL)

# 如果已存在，先刪掉重建（避免測試時混到舊資料）
if client.collection_exists(COLLECTION_NAME):
    client.delete_collection(COLLECTION_NAME)

client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config={
        "dense": models.VectorParams(
            size=DENSE_VECTOR_SIZE,
            distance=models.Distance.COSINE,
        ),
    },
    sparse_vectors_config={
        "sparse": models.SparseVectorParams(
            modifier=models.Modifier.IDF
        )
    },
)

print(f"✅ Qdrant collection ready: {COLLECTION_NAME}")


✅ Qdrant collection ready: cw04_hybrid_rerank


# Step 2-A：讀取 chunks_semantic.jsonl 成為 chunks list

In [46]:
# %%
import json
from pathlib import Path

CHUNKS_JSONL = Path("./chunks_semantic.jsonl")

chunks = []
with CHUNKS_JSONL.open("r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        chunks.append(json.loads(line))

print(f"✅ Loaded chunks: {len(chunks)}")
print("Example keys:", list(chunks[0].keys()))
print("Example:", chunks[0]["id"], chunks[0]["source"], chunks[0]["text"][:80])


✅ Loaded chunks: 36
Example keys: ['id', 'source', 'chunk_id', 'start_token', 'end_token', 'text', 'source_path', 'tokenizer', 'method', 'max_tokens', 'min_tokens', 'sim_threshold']
Example: data_01.txt::0 data_01.txt 今天是2月2日星期一，台中市迎來了一個涼爽而舒適的清晨。
凌晨時分，氣溫約在攝氏16度左右，空氣中帶著冬季特有的清新感。
隨著太陽逐漸升起，天空呈現出淡藍色的基


# Step 2-B：批次呼叫 Embedding API

In [47]:
# %%
import requests

EMBED_API_URL = "https://ws-04.wade0426.me/embed"
TASK_DESCRIPTION = "檢索技術文件"

def get_embeddings(texts):
    r = requests.post(
        EMBED_API_URL,
        json={
            "texts": texts,
            "task_description": TASK_DESCRIPTION,
            "normalize": True,
        },
        timeout=120
    )
    r.raise_for_status()
    return r.json()["embeddings"]

texts = [c["text"] for c in chunks]

BATCH = 16
embeddings = []
for i in range(0, len(texts), BATCH):
    batch_texts = texts[i:i+BATCH]
    embeddings.extend(get_embeddings(batch_texts))

print(f"✅ Embeddings: {len(embeddings)}")
print("Dim:", len(embeddings[0]))


✅ Embeddings: 36
Dim: 4096


# Step 2-C：Upsert 到 Qdrant（dense + sparse/BM25）

In [48]:
# %%
from qdrant_client import QdrantClient, models
import uuid

QDRANT_URL = "http://localhost:6333"
COLLECTION_NAME = "cw04_hybrid_rerank"

client = QdrantClient(url=QDRANT_URL)

points = []
for c, emb in zip(chunks, embeddings):
    qdrant_id = uuid.uuid4().hex  # ✅ 合法 id（UUID）

    points.append(
        models.PointStruct(
            id=qdrant_id,
            vector={
                "dense": emb,
                "sparse": models.Document(text=c["text"], model="Qdrant/bm25"),
            },
            payload={
                "text": c["text"],
                "source_file": c.get("source", ""),
                "chunk_id": c.get("chunk_id", None),
                "start_token": c.get("start_token", None),
                "end_token": c.get("end_token", None),
                "chunk_uid": c.get("id", ""),  # ✅ 把 data_01.txt::0 留下來
            },
        )
    )

client.upsert(collection_name=COLLECTION_NAME, points=points)
print(f"✅ Upserted {len(points)} points into {COLLECTION_NAME}")


✅ Upserted 36 points into cw04_hybrid_rerank


# Step 3-A：先寫一個 Hybrid Search 函式（只負責拿候選 chunks）

In [49]:
# %%
from qdrant_client import QdrantClient, models
import requests

QDRANT_URL = "http://localhost:6333"
COLLECTION_NAME = "cw04_hybrid_rerank"

EMBED_API_URL = "https://ws-04.wade0426.me/embed"
TASK_DESCRIPTION = "檢索技術文件"

client = QdrantClient(url=QDRANT_URL)

def embed_query(query: str):
    r = requests.post(
        EMBED_API_URL,
        json={"texts": [query], "task_description": TASK_DESCRIPTION, "normalize": True},
        timeout=60
    )
    r.raise_for_status()
    return r.json()["embeddings"][0]

def hybrid_retrieve(query: str, initial_limit: int = 20):
    q_dense = embed_query(query)

    res = client.query_points(
        collection_name=COLLECTION_NAME,
        # 兩路 prefetch：sparse(BM25) + dense(embedding)
        prefetch=[
            models.Prefetch(
                query=models.Document(text=query, model="Qdrant/bm25"),
                using="sparse",
                limit=initial_limit,
            ),
            models.Prefetch(
                query=q_dense,
                using="dense",
                limit=initial_limit,
            ),
        ],
        # 用 RRF 融合（學長示範的融合方式）
        query=models.FusionQuery(fusion=models.Fusion.RRF),
        limit=initial_limit,
    )

    hits = []
    for p in res.points:
        payload = p.payload or {}
        hits.append({
            "point_id": p.id,
            "score": p.score,
            "text": payload.get("text", ""),
            "source_file": payload.get("source_file", ""),
            "chunk_uid": payload.get("chunk_uid", ""),
        })
    return hits


# Step 3-B：先用一題測試（確認 retrieval 正常）

In [50]:
# %%
test_query = "LiteRT 更新有哪些重點？"
hits = hybrid_retrieve(test_query, initial_limit=10)

print(f"✅ Retrieved {len(hits)} candidates")
for i, h in enumerate(hits[:5], 1):
    print(f"\n[{i}] score={h['score']:.4f} source={h['source_file']} chunk_uid={h['chunk_uid']}")
    print(h["text"][:160])


✅ Retrieved 10 candidates

[1] score=0.5000 source=data_05.txt chunk_uid=data_05.txt::5
LiteRT的訴求是把差異整合進同一套機制，讓開發者以較一致的方法啟用NPU加速，並在裝置不支援或條件不足時，仍能自動改用GPU或CPU維持可用性。

[2] score=0.3333 source=data_05.txt chunk_uid=data_05.txt::1
LiteRT技術堆疊從TensorFlow Lite的基礎往前推進，過去TensorFlow Lite主要服務傳統機器學習推論，而LiteRT的定位則是接手新一代裝置端AI需求，包含更廣泛的硬體加速與跨平臺部署。

[3] score=0.2500 source=data_05.txt chunk_uid=data_05.txt::0
Google更新裝置端推論框架LiteRT，宣布在Google I/O 2025預告的進階硬體加速能力，已正式納入LiteRT產品堆疊並對開發者開放。
這次更新把GPU與NPU的加速流程補齊，其中GPU支援從I/O 2025當時先在Android導入的路徑，擴大到Android、iOS、macOS、Windows、Li

[4] score=0.2000 source=data_05.txt chunk_uid=data_05.txt::2
LiteRT的GPU加速支援範圍涵蓋Android、iOS、macOS、Windows、Linux與Web，並透過ML Drift這套下一代GPU引擎，串接OpenCL、OpenGL、Metal與WebGPU等後端。
在Android裝置上，LiteRT會在可用時優先採用OpenCL以取得較高效能，必要時退回OpenG

[5] score=0.1667 source=data_05.txt chunk_uid=data_05.txt::7
同時，LiteRT提供AOT與裝置端JIT兩種編譯策略，讓開發者在啟動速度與首次執行成本之間做取捨。
LiteRT維持以.tflite格式作為跨平臺部署的共同基礎，開發者可利用PyTorch、TensorFlow與JAX等常見訓練框架轉換模型，讓不同來源的模型都能接上同一套裝置端推論與硬體加速能力，降低因訓練框架不同而

# Step 4-A：載入 Reranker 模型

In [51]:
# %%
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# TODO: 換成你剛剛在 WSL 終端機確認存在的那條路徑
MODEL_DIR = "/home/randy/Qwen3-Reranker-0.6B"

tokenizer_rerank = AutoTokenizer.from_pretrained(
    MODEL_DIR,
    trust_remote_code=True,
    local_files_only=True,
)

model_rerank = AutoModelForCausalLM.from_pretrained(
    MODEL_DIR,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto",
    trust_remote_code=True,
    local_files_only=True,
)
model_rerank.eval()

print("✅ Reranker loaded. CUDA:", torch.cuda.is_available())
print("Model path:", MODEL_DIR)


✅ Reranker loaded. CUDA: True
Model path: /home/randy/Qwen3-Reranker-0.6B


# Step 4-B：定義 ReRank 打分函式

In [52]:
# %%
import math

@torch.inference_mode()
def rerank_score_yes_prob(query: str, doc: str) -> float:
    prompt = (
        "You are a relevance judge.\n"
        "Given a query and a document, answer 'yes' if the document is relevant to the query, otherwise answer 'no'.\n\n"
        f"Query: {query}\n"
        f"Document: {doc}\n"
        "Answer:"
    )

    inputs = tokenizer_rerank(prompt, return_tensors="pt", truncation=True, max_length=2048)
    inputs = {k: v.to(model_rerank.device) for k, v in inputs.items()}

    out = model_rerank(**inputs)
    logits = out.logits[0, -1]  # 最後一個 token 的 logits

    # 取得 " yes" / " no" 的 token id（包含前導空白比較常見）
    yes_ids = tokenizer_rerank.encode(" yes", add_special_tokens=False)
    no_ids  = tokenizer_rerank.encode(" no", add_special_tokens=False)

    # 保守處理：取第一個 token id（通常就夠用）
    yes_id = yes_ids[0]
    no_id = no_ids[0]

    yes_logit = logits[yes_id].float().item()
    no_logit  = logits[no_id].float().item()

    # softmax over {yes, no}
    m = max(yes_logit, no_logit)
    yes_p = math.exp(yes_logit - m) / (math.exp(yes_logit - m) + math.exp(no_logit - m))
    return float(yes_p)


# Step 4-C：對 Step 3 的 hits 做 rerank，輸出 Top-K

In [53]:
# %%
def rerank_hits(query: str, hits: list, final_top_k: int = 5):
    scored = []
    for h in hits:
        s = rerank_score_yes_prob(query, h["text"])
        scored.append({**h, "rerank_score": s})
    scored.sort(key=lambda x: x["rerank_score"], reverse=True)
    return scored[:final_top_k], scored

# 用你剛剛 Step 3 的同一題測試
query = test_query  # "LiteRT 更新有哪些重點？"
topk, all_scored = rerank_hits(query, hits, final_top_k=5)

print("✅ Rerank Top-5:")
for i, h in enumerate(topk, 1):
    print(f"\n[{i}] rerank={h['rerank_score']:.4f} (rrf={h['score']:.4f}) source={h['source_file']} uid={h['chunk_uid']}")
    print(h["text"][:160])


✅ Rerank Top-5:

[1] rerank=0.9986 (rrf=0.2500) source=data_05.txt uid=data_05.txt::0
Google更新裝置端推論框架LiteRT，宣布在Google I/O 2025預告的進階硬體加速能力，已正式納入LiteRT產品堆疊並對開發者開放。
這次更新把GPU與NPU的加速流程補齊，其中GPU支援從I/O 2025當時先在Android導入的路徑，擴大到Android、iOS、macOS、Windows、Li

[2] rerank=0.9951 (rrf=0.1667) source=data_05.txt uid=data_05.txt::7
同時，LiteRT提供AOT與裝置端JIT兩種編譯策略，讓開發者在啟動速度與首次執行成本之間做取捨。
LiteRT維持以.tflite格式作為跨平臺部署的共同基礎，開發者可利用PyTorch、TensorFlow與JAX等常見訓練框架轉換模型，讓不同來源的模型都能接上同一套裝置端推論與硬體加速能力，降低因訓練框架不同而

[3] rerank=0.9868 (rrf=0.3333) source=data_05.txt uid=data_05.txt::1
LiteRT技術堆疊從TensorFlow Lite的基礎往前推進，過去TensorFlow Lite主要服務傳統機器學習推論，而LiteRT的定位則是接手新一代裝置端AI需求，包含更廣泛的硬體加速與跨平臺部署。

[4] rerank=0.9797 (rrf=0.5000) source=data_05.txt uid=data_05.txt::5
LiteRT的訴求是把差異整合進同一套機制，讓開發者以較一致的方法啟用NPU加速，並在裝置不支援或條件不足時，仍能自動改用GPU或CPU維持可用性。

[5] rerank=0.9325 (rrf=0.1429) source=data_05.txt uid=data_05.txt::3
Google表示，在多種模型的平均情境下，LiteRT的GPU效能平均約比既有的TensorFlow Lite GPU委派快1.4倍。
LiteRT的效能改善，來自推論從輸入到輸出的整體等待時間縮短，Google讓裝置端推論更少依賴CPU執行額外的等待與資料處

# Step 5-A：讀取 Prompt 模板

In [54]:
# %%
from docx import Document

PROMPT_DOCX_PATH = "Prompt.txt.docx"  # 若你檔名不同就改這裡

doc = Document(PROMPT_DOCX_PATH)
PROMPT_TEMPLATE = "\n".join([p.text for p in doc.paragraphs]).strip()

print("✅ Prompt loaded. length =", len(PROMPT_TEMPLATE))
print(PROMPT_TEMPLATE[:200], "...")


✅ Prompt loaded. length = 241
你是一位專業的知識庫助手。請嚴格根據以下提供的【參考資訊】來回答用戶的問題。

### 規則：
1. **必須**只依賴【參考資訊】中的內容回答。
2. 如果【參考資訊】中沒有足夠的資訊來回答問題，請直接回答：「抱歉，根據目前的資料庫，我無法回答這個問題。」，**絕對不要**憑空捏造或使用你的外部知識。
3. 回答應簡潔、準確且專業。

### 參考資訊：
"""
{{context_str}}
 ...


# Step 5-B：把 Top-K chunks 組成 context_str

In [55]:
# %%
def build_context(topk_hits):
    blocks = []
    for i, h in enumerate(topk_hits, 1):
        blocks.append(
            f"[{i}] source_file={h['source_file']} chunk_uid={h.get('chunk_uid','')}\n{h['text']}"
        )
    return "\n\n".join(blocks)

context_str = build_context(topk)
chosen_source_file = topk[0]["source_file"] if topk else ""

print("✅ Context built. chars =", len(context_str))
print("Chosen source_file:", chosen_source_file)
print(context_str[:300], "...")


✅ Context built. chars = 966
Chosen source_file: data_05.txt
[1] source_file=data_05.txt chunk_uid=data_05.txt::0
Google更新裝置端推論框架LiteRT，宣布在Google I/O 2025預告的進階硬體加速能力，已正式納入LiteRT產品堆疊並對開發者開放。
這次更新把GPU與NPU的加速流程補齊，其中GPU支援從I/O 2025當時先在Android導入的路徑，擴大到Android、iOS、macOS、Windows、Linux與Web，讓裝置端AI推論在行動端、桌面與網頁之間更一致。

[2] source_file=data_05.txt chunk_uid=data_05.txt::7
 ...


# Step 5-C：呼叫 LLM API

In [56]:
# %%
import requests

LLM_API_URL = "https://ws-03.wade0426.me/v1/chat/completions"
LLM_MODEL = "/models/gpt-oss-120b"

def render_prompt(template: str, context_str: str, query_str: str) -> str:
    return (
        template
        .replace("{{context_str}}", context_str)
        .replace("{{query_str}}", query_str)
    )

def call_llm(prompt: str, temperature: float = 0.0, max_tokens: int = 512) -> str:
    payload = {
        "model": LLM_MODEL,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens,
    }
    r = requests.post(LLM_API_URL, json=payload, timeout=180)
    r.raise_for_status()
    data = r.json()
    return data["choices"][0]["message"]["content"]

final_prompt = render_prompt(PROMPT_TEMPLATE, context_str=context_str, query_str=query)

answer = call_llm(final_prompt, temperature=0.0, max_tokens=512)

print("✅ Answer:\n")
print(answer)


✅ Answer:

LiteRT 更新的重點包括：

1. **硬體加速擴充**  
   - 完成 GPU 與 NPU 的加速流程，支援平台從 Android 擴展至 iOS、macOS、Windows、Linux 與 Web，讓裝置端 AI 推論在行動、桌面與網頁間更一致。

2. **編譯策略**  
   - 同時提供 **AOT（Ahead‑Of‑Time）** 與 **裝置端 JIT（Just‑In‑Time）** 兩種編譯方式，讓開發者可在啟動速度與首次執行成本之間自行取捨。

3. **跨框架模型部署**  
   - 仍以 **.tflite** 為跨平台部署的共同基礎，支援從 PyTorch、TensorFlow、JAX 等常見訓練框架轉換模型，降低因訓練框架不同而產生的部署分歧。

4. **統一加速機制與自動回退**  
   - 以單一機制整合差異，讓開發者以一致方式啟用 NPU 加速；若裝置不支援或條件不足，系統會自動回退至 GPU 或 CPU，確保可用性。

5. **效能提升**  
   - 在多種模型的平均情境下，LiteRT 的 GPU 效能約比既有的 TensorFlow Lite GPU 委派快 **1.4 倍**。  
   - 效能改善來自整體推論等待


# Step 6-A — 讀取 questions.csv，確認欄位與建立輸出欄位

In [57]:
# %%
import pandas as pd
from pathlib import Path

QUESTIONS_PATH = Path("./questions_answer.csv")
df = pd.read_csv(QUESTIONS_PATH)

required_cols = ["題目_ID", "題目", "標準答案", "來源文件"]
missing = [c for c in required_cols if c not in df.columns]
assert not missing, f"❌ questions.csv 缺少欄位: {missing}"

# 建議新增輸出欄位（不破壞原本資料）
if "模型回答" not in df.columns:
    df["模型回答"] = ""
if "命中來源文件" not in df.columns:
    df["命中來源文件"] = ""

print("✅ Loaded questions.csv rows =", len(df))
print("Columns:", list(df.columns))
df.head(3)


✅ Loaded questions.csv rows = 10
Columns: ['題目_ID', '題目', '標準答案', '來源文件', '模型回答', '命中來源文件']


,題目_ID,題目,標準答案,來源文件,模型回答,命中來源文件
0,1,文中提到台中市2月2日當天的氣溫變化範圍為何？,凌晨約攝氏16度左右，白天最高溫約23度。,data_01.txt,,
1,2,Google Cloud 推出的 N4A 虛擬機器採用哪一款處理器？,Google Axion 處理器（Arm 架構）。,data_04.txt,,
2,3,根據林氏璧的指出，日本這波流感主要流行哪兩種病毒型別？其占比為何？,以A型H3為主（占74%），混著B型流感（占26%）。,data_03.txt,,


# Step 6-B — 定義「單題完整流程」：Hybrid → ReRank → LLM

In [58]:
# %%
import time

# ---- 你前面已經有的函式/物件，這裡假設已存在： ----
# client, COLLECTION_NAME
# hybrid_retrieve(query, initial_limit)
# rerank_hits(query, hits, final_top_k)  -> (topk, all_scored)
# PROMPT_TEMPLATE
# render_prompt(template, context_str, query_str)
# call_llm(prompt, temperature, max_tokens)

def build_context(topk_hits):
    blocks = []
    for i, h in enumerate(topk_hits, 1):
        blocks.append(
            f"[{i}] source_file={h['source_file']} chunk_uid={h.get('chunk_uid','')}\n{h['text']}"
        )
    return "\n\n".join(blocks)

def answer_one_question(query: str,
                        initial_limit: int = 20,
                        final_top_k: int = 5,
                        llm_max_tokens: int = 512,
                        sleep_sec: float = 0.2):
    """
    單題 end-to-end:
    Hybrid retrieval -> rerank -> context -> LLM answer
    """
    try:
        # 1) Hybrid retrieval
        hits = hybrid_retrieve(query, initial_limit=initial_limit)
        if not hits:
            return "抱歉，根據目前的資料庫，我無法回答這個問題。", ""

        # 2) ReRank
        topk, _ = rerank_hits(query, hits, final_top_k=final_top_k)
        if not topk:
            return "抱歉，根據目前的資料庫，我無法回答這個問題。", ""

        # 3) Build context
        context_str = build_context(topk)
        chosen_source_file = topk[0].get("source_file", "")

        # 4) LLM
        final_prompt = render_prompt(PROMPT_TEMPLATE, context_str=context_str, query_str=query)
        answer = call_llm(final_prompt, temperature=0.0, max_tokens=llm_max_tokens)

        # 節流（避免 API 太密）
        if sleep_sec > 0:
            time.sleep(sleep_sec)

        return answer, chosen_source_file

    except Exception as e:
        # 不讓整批中斷
        return f"【系統錯誤】{type(e).__name__}: {e}", ""


# Step 6-C — 批次跑全題目並輸出 CSV

In [59]:
# %%
from tqdm import tqdm

OUTPUT_PATH = Path("./questions_answer_model.csv")

# 先試跑幾題（你可以先用 3 或 5，確認沒問題再全跑）
DRY_RUN = 10  # 改成 None 代表全跑

initial_limit = 20
final_top_k = 5

rows = df.index.tolist()
if DRY_RUN is not None:
    rows = rows[:DRY_RUN]

for idx in tqdm(rows):
    q = str(df.loc[idx, "題目"]).strip()
    if not q:
        continue

    ans, src = answer_one_question(
        q,
        initial_limit=initial_limit,
        final_top_k=final_top_k,
        llm_max_tokens=256,
        sleep_sec=1.0,
    )

    df.loc[idx, "模型回答"] = ans
    df.loc[idx, "命中來源文件"] = src

# 如果你要「直接回填來源文件欄位」，取消註解這行：
# df.loc[rows, "來源文件"] = df.loc[rows, "命中來源文件"]

df.to_csv(OUTPUT_PATH, index=False, encoding="utf-8-sig")
print(f"✅ Saved: {OUTPUT_PATH.resolve()}")
df.loc[rows, ["題目_ID", "題目", "命中來源文件", "模型回答"]].head(len(rows))


100%|██████████| 10/10 [02:23<00:00, 14.35s/it]

✅ Saved: /home/randy/Day6_hw/CW/04/questions_answer_model.csv


,題目_ID,題目,命中來源文件,模型回答
0,1,文中提到台中市2月2日當天的氣溫變化範圍為何？,data_01.txt,根據資料，台中市2月2日的氣溫在當天的變化範圍約為 **16 °C（凌晨）至 23 °C（白...
1,2,Google Cloud 推出的 N4A 虛擬機器採用哪一款處理器？,data_04.txt,Google Cloud 的 N4A 虛擬機器採用自行研發的 **Google Axion ...
2,3,根據林氏璧的指出，日本這波流感主要流行哪兩種病毒型別？其占比為何？,data_03.txt,None
3,4,LiteRT 的 GPU 加速效能平均比原本的 TensorFlow Lite GPU 委派...,data_05.txt,根據 Google 的測試，LiteRT 的 GPU 加速效能在多種模型的平均情境下，約比既...
4,5,台灣南部與北部的降雨特徵有何主要差異？,data_02.txt,根據資料，台灣北部與南部的降雨特徵主要差異如下：\n\n| 項目 | 北部 | 南部 |\n...
5,6,Google Cloud 的 N4A 虛擬機器在今年1月底擴增的區域中，包含哪一個亞洲地區？,data_04.txt,根據資料，擴增的區域中包含亞洲地區 **asia‑southeast1（新加坡）**。
6,7,LiteRT 目前支援哪些平台的 GPU 加速？,data_05.txt,根據目前的資料，LiteRT 的 GPU 加速支援以下平台：\n\n- Android \...
7,8,根據日本流感疫情報告，東京目前比較流行的區域有哪些？,data_03.txt,根據日本流感疫情報告，東京目前比較流行的區域包括：\n\n- 八王子市 \n- 町田市 ...
8,9,針對台中市2月2日的天氣，文中建議民眾該如何穿著？,data_01.txt,根據資料，建議在2月2日於台中市外出時穿著輕薄的長袖衣物，並隨身攜帶一件薄外套，以因應早晚的溫差。
9,10,台灣冬季東北部迎風面多雨，形成「竹風蘭雨」的特色，主要是受到什麼風的影響？,data_02.txt,主要受到冬季吹來的**東北季風**影響。
